# CNN

## 1. Settings

### 1) import requires libraries

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as otm
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as trfm
from torch.utils.data import DataLoader
from torch.autograd import Variable

### 2) set hyperparameters

In [ ]:
batch_size = 128
lr = 0.0002
num_epoch = 20

is_gpu = True

## 2. Data

### 1) download data

In [ ]:
mnist_train = dset.MNIST("./", train=True, transform=trfm.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST("./", train=False, transform=trfm.ToTensor(), target_transform=None, download=True)

In [ ]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
mnist_train.__getitem__(0)[0].size(), mnist_train.__len__()

In [ ]:
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

## 3.Model & Optimizer

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(512*7*7, 400),
            nn.ReLU(),
            nn.Linear(400, 10)
        )
    
    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        return out
    
cnn = CNN()#.cuda()

loss_func = nn.CrossEntropyLoss()
optimizer = otm.SGD(cnn.parameters(), lr = lr)

## 4. Train

In [ ]:
for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader):
        optimizer.zero_grad() # 언제나 otm은 제로로 리셋을 시켜줘야합니다.
        
        image = Variable(image)#.cuda()
        label = Variable(label)#.cuda()
        

        result = cnn.forward(image)
        loss = loss_func(result, image)
        loss.backward()
        optimizer.step()
        
        if j % 100 == 0:
            print(loss)

In [ ]:
result.size()

In [ ]:
image.size()

In [ ]:
# Test with test data
correct = 0
total = 0

for image, label in test_loader:
    image = Variable(image)#.cuda()
    result = Variable(label)#.cuda()
    
    output = cnn.forward(image)
    _, predicted = torch.max(output, 1)
    
    total += label.size(0)
    correct += (predicted == label).sum().float()
    
print(100 * correct / total)